In [ ]:
!pip install argilla -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.1 MB/s eta 0:00:00


In [ ]:
print(rg.__version__)

2.7.0


In [ ]:
from argilla import client

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `chatbot` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `chatbot`


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import argilla as rg  # Argilla library for logging/annotating data
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def fetch_website_data(url):
    headers = {
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/91.0.4472.124 Safari/537.36")
    }
    try:
        url_clean = url.strip()
        logger.info(f"Fetching URL: {url_clean}")
        response = requests.get(url_clean, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        title = soup.title.string.strip() if soup.title and soup.title.string else "No Title"
        description_tag = soup.find("meta", attrs={"name": "description"})
        description = (description_tag["content"].strip()
                       if description_tag and description_tag.get("content")
                       else "No Description")
        content = soup.get_text(separator=" ", strip=True)
        return {
            "Website": url_clean,
            "Title": title,
            "Content": content,
            "Metadata": description
        }
    except Exception as e:
        logger.error(f"Error for {url}: {e}")
        return {"Website": url.strip(), "Title": None, "Content": None, "Metadata": None}

def main():
    urls = [
        "https://www.fiscozen.it/guide/partita-iva-a-cosa-serve/",
        "https://www.agenziaentrate.gov.it/portale/iva-regole-generali-aliquote-esenzioni-pagamento/infogen-iva-regole-generali-aliquote-esenzioni-pagamento"
    ]
    data_records = [fetch_website_data(url) for url in urls]
    dataset_name = "web_data_collection"

    try:
        # Initialize the Argilla client
        client = rg.Argilla(
            api_url="https://glopops-argilla-space.hf.space",
            api_key="ai4f9sThzZtTo5RZCCqwEzSNyv_2Jwp6N4HMLWhWlSbN5fsUcFD-P-c9cCPRUZwqKOB0aaXkY8WE4xsQUWtqx7_ztt5qMXf_dLifZR8rKu4"  # Replace with your actual API key
        )

        # Define dataset settings
        settings = rg.Settings(
            fields=[
                rg.TextField(name="Website"),
                rg.TextField(name="Title"),
                rg.TextField(name="Content"),
                rg.TextField(name="Metadata")
            ],
            questions=[
                rg.TextQuestion(name="Annotation", description="Annotate the content.")
            ],
            guidelines="Please provide annotations for the website content."
        )

        # Create a new dataset
        dataset = rg.Dataset(
            name=dataset_name,
            settings=settings,
            client=client
        )
        dataset.create()
        logger.info(f"Created new dataset: {dataset_name}")

        # Prepare records
        records = [
            rg.Record(
                fields={
                    "Website": record["Website"],
                    "Title": record["Title"],
                    "Content": record["Content"],
                    "Metadata": record["Metadata"]
                }
            ) for record in data_records
        ]

        # Log records to the dataset
        dataset.records.log(records=records)
        logger.info(f"Logged {len(records)} records to dataset: {dataset_name}")

        # Load records from the dataset
        loaded_records = list(dataset.records)

        # Convert to DataFrame
        df = pd.DataFrame([record.fields for record in loaded_records])
        df.to_csv("web_data.csv", index=False)
        logger.info("Data successfully saved to web_data.csv")
        return df

    except Exception as e:
        logger.error(f"An error occurred: {e}")
        return None

if __name__ == "__main__":
    data_frame = main()


ERROR:__main__:An error occurred: Argilla SDK error: ArgillaCredentialsError: Credentials (api_key and/or api_url) are invalid
